In [1]:
import initialize

In [3]:

# 1) create the optimization algorithm
# 2) decide on th


# x*1 + y* 2
# lambda x, y: x * 1 + y * 2
# Constaint = LTE(x=100, y=50) & GTE(x=0, y=0)
# genetic = Genetic()
# fitter.fit(func, constraint, x=20, y=30, const=['x'], penalty=-inf)
# fitter.step( state=state)
# 1) create the individual -> only x
# 2) populate
# 3) If not first
#    get children
#    cross over
#    elitism
# 3) mutate
# 4) assess
# 5) yield assessments

In [2]:
from zenkai import tansaku

x = 20
y = 30
populator = tansaku.RepeatPopulator(10)


objective = tansaku.FuncObjective(
    lambda x, y: 2 * x + 2.5 * y 
)

# how to convert to float
individual = tansaku.Individual(x=20.0, y=30.0)
population = populator(individual)

mutator = tansaku.GaussianMutator(0.2, 0.0)
population = mutator(population)

population = tansaku.Population(
    x=population['x'].clamp(0, 30),
    y=population['y'].clamp(0, 20)
)



print(population['x'])

# # Add a "Lambda" constraint
# LC(['x', 't'], lambda x, t: 2 * x + 3 * t < 4)
# LC() + 

objective = tansaku.FuncObjective(lambda x, y: 3 * x + 5 * y, penalty=1000)
assessor = tansaku.ObjectivePopAssessor(objective, ['x', 'y'])

population = assessor(population)

divider = tansaku.FitnessProportionateDivider(6)
parents1, parents2 = divider(population)

print(parents1['x'], parents2['x'])
crossover = tansaku.SmoothCrossOverBreeder()
children =crossover(parents1, parents2)
print(children['x'])

elitism = tansaku.KBestElitism(2)
children2 = elitism(population, children)
print(children2['x'])


tensor([19.9871, 20.0184, 20.1183, 20.0529, 20.2111, 20.4001, 20.0721, 20.0011,
        20.2667, 19.6715])
objective
tensor([159.9612, 160.0551, 160.3550, 160.1588, 160.6334, 161.2003, 160.2162,
        160.0033, 160.8001, 159.0146])
tensor([159.9612, 160.0551, 160.3550, 160.1588, 160.6334, 161.2003, 160.2162,
        160.0033, 160.8001, 159.0146])
tensor([20.4001, 20.0184, 20.2667, 19.6715, 20.0721, 20.0529]) tensor([20.2111, 20.4001, 20.0529, 20.0529, 20.1183, 20.0721])
tensor([20.2916, 20.0200, 20.2665, 19.9762, 20.1081, 20.0648])
tensor([19.6715, 19.9871, 20.2916, 20.0200, 20.2665, 19.9762, 20.1081, 20.0648])


In [3]:
import typing
from zenkai import tansaku
from zenkai.kaku import Assessment, State
import torch
from functools import partial

class GeneticAlgorithm(tansaku.Itadaki):

    def __init__(self) -> None:
        super().__init__()
        self.divider = tansaku.FitnessProportionateDivider(8)
        self.elitism = tansaku.KBestElitism(2)
        self.crossover = tansaku.SmoothCrossOverBreeder()
        self.mutator = tansaku.GaussianMutator(0.5, 0.0)
        self.populator = tansaku.RepeatPopulator(10)
        self.iterations = 100

    def optim_iter(self, objective: tansaku.Objective, state: State = None, **kwargs) -> typing.Iterator[Assessment]:
        
        state = state or State()
        my_state = state.mine((self, objective))
        assessor = tansaku.ObjectivePopAssessor(objective, ['x', 'y'])

        i = my_state.get_or_set('i', 0)
        if i == 0:
            individual = tansaku.Individual(**kwargs)
            population = self.populator(individual, state)
            population = self.mutator(population, state)
        else:
            population = my_state.get('population')

        while i < self.iterations:
            
            if i > 0:
                parents1, parents2 = self.divider(population, state)
                
                children = self.crossover(parents1, parents2, state)
                children = self.mutator(children, state)
                population = self.elitism(population, children, state)
            
            population = population.apply(partial(torch.clamp, min=0))

            population = assessor(population, state)
            my_state['population'] = population
            yield population.stack_assessments()
            i += 1


objective = tansaku.FuncObjective(
    lambda x, y: torch.abs(2 * x + 3 * y), constraint=tansaku.LTE(y=5, x=10) + tansaku.GTE(y=0, x=0), penalty=0, maximize=True
)


genetic_algorithm = GeneticAlgorithm()

for assessment in genetic_algorithm.optim_iter(objective, x=3.0, y=4.0):
    print(assessment)

    

tensor([3.5103, 2.9007, 4.1692, 3.1180, 3.1168, 2.7948, 3.2687, 3.1598, 2.8878,
        2.0694]) tensor([4.3859, 4.1083, 4.7605, 3.5392, 3.9393, 4.1118, 3.8909, 3.7738, 3.7663,
        3.3328])

        Assessment(maximize: True, value: tensor([20.1783, 18.1263, 22.6197, 16.8535, 18.0515, 17.9250, 18.2100, 17.6408,
        17.0747, 14.1370]))
        
tensor([4.1692, 3.5103, 3.7421, 3.4145, 2.2916, 2.9261, 3.4894, 3.3802, 4.2302,
        3.7443]) tensor([4.7605, 4.3859, 4.3757, 3.5977, 2.7381, 3.8951, 4.8844, 4.2130, 3.7139,
        3.8733])

        Assessment(maximize: True, value: tensor([22.6197, 20.1783, 20.6115, 17.6223, 12.7975, 17.5373, 21.6320, 19.3993,
        19.6022, 19.1085]))
        
tensor([4.1692, 3.4894, 3.7010, 3.7149, 3.7175, 3.8314, 3.2284, 3.7103, 4.1044,
        3.4062]) tensor([4.7605, 4.8844, 4.7451, 4.8502, 4.8390, 4.8712, 5.0078, 4.7434, 3.4766,
        4.4499])

        Assessment(maximize: True, value: tensor([22.6197, 21.6320, 21.6373, 21.9806, 21.9520, 22